# darkflowの構築

In [0]:
!git clone https://github.com/thtrieu/darkflow.git

Cloning into 'darkflow'...
remote: Enumerating objects: 2706, done.
remote: Total 2706 (delta 0), reused 0 (delta 0), pack-reused 2706
Receiving objects: 100% (2706/2706), 32.98 MiB | 47.03 MiB/s, done.
Resolving deltas: 100% (1760/1760), done.


In [0]:
%cd darkflow/

/content/darkflow


In [0]:
!python3 setup.py build_ext --inplace

Compiling darkflow/cython_utils/nms.pyx because it changed.
Compiling darkflow/cython_utils/cy_yolo2_findboxes.pyx because it changed.
Compiling darkflow/cython_utils/cy_yolo_findboxes.pyx because it changed.
[1/3] Cythonizing darkflow/cython_utils/cy_yolo2_findboxes.pyx
/usr/local/lib/python3.6/dist-packages/Cython/Compiler/Main.py:367: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/darkflow/darkflow/cython_utils/cy_yolo2_findboxes.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
[2/3] Cythonizing darkflow/cython_utils/cy_yolo_findboxes.pyx
/usr/local/lib/python3.6/dist-packages/Cython/Compiler/Main.py:367: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/darkflow/darkflow/cython_utils/cy_yolo_findboxes.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
[3/3] Cythonizing darkflow/cython_utils/nms.pyx
/usr/

In [0]:
# !cat darkflow/utils/loader.py

In [0]:
!sed -i 's/self.offset = 16/self.offset = 20/g' darkflow/utils/loader.py
%cd ..

/content


# データの配置

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!mkdir input

In [0]:
!cp /content/gdrive/My\ Drive/Python3/fujifilm/data/images.zip input/
!cp /content/gdrive/My\ Drive/Python3/fujifilm/data/train_anns.zip input/
!cp /content/gdrive/My\ Drive/Python3/fujifilm/data/test_images.zip input/
%cd input
!unzip images.zip
!unzip train_anns.zip
!unzip test_images.zip
%cd ..
!mv input/images input/train_images
!cp /content/gdrive/My\ Drive/Python3/fujifilm/data/test0.2.txt  .

/content/input
Archive:  images.zip
   creating: images/
  inflating: images/process.py       
  inflating: images/train_id0003_no0031.jpg  
  inflating: images/train_id0003_no0032.jpg  
  inflating: images/train_id0003_no0033.jpg  
  inflating: images/train_id0003_no0034.jpg  
  inflating: images/train_id0003_no0035.jpg  
  inflating: images/train_id0003_no0036.jpg  
  inflating: images/train_id0003_no0037.jpg  
  inflating: images/train_id0003_no0038.jpg  
  inflating: images/train_id0003_no0046.jpg  
  inflating: images/train_id0003_no0047.jpg  
  inflating: images/train_id0003_no0048.jpg  
  inflating: images/train_id0003_no0049.jpg  
  inflating: images/train_id0003_no0050.jpg  
  inflating: images/train_id0003_no0051.jpg  
  inflating: images/train_id0003_no0052.jpg  
  inflating: images/train_id0003_no0062.jpg  
  inflating: images/train_id0003_no0063.jpg  
  inflating: images/train_id0003_no0064.jpg  
  inflating: images/train_id0003_no0065.jpg  
  inflating: images/train_id000

In [0]:
%cd darkflow/
!cp /content/gdrive/My\ Drive/Python3/fujifilm/data/myyolov2-voc.cfg cfg/
!cp /content/gdrive/My\ Drive/Python3/fujifilm/data/obj.names labels.txt
!mkdir backup
!cp /content/gdrive/My\ Drive/Python3/fujifilm/data/weights/myyolov2-voc-20000.weights backup/myyolov2-voc-20000.weights

/content/darkflow


# トレーニングデータで精度を確認
## ※学習データはすべてtrainingに利用している

In [0]:
from darkflow.net.build import TFNet
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import difflib
import os
import cv2
import re
%config InlineBackend.figure_format = 'retina'

W0620 09:55:20.850860 140212654778240 deprecation_wrapper.py:119] From /content/darkflow/darkflow/net/build.py:15: The name tf.train.RMSPropOptimizer is deprecated. Please use tf.compat.v1.train.RMSPropOptimizer instead.

W0620 09:55:20.852367 140212654778240 deprecation_wrapper.py:119] From /content/darkflow/darkflow/net/build.py:16: The name tf.train.AdadeltaOptimizer is deprecated. Please use tf.compat.v1.train.AdadeltaOptimizer instead.

W0620 09:55:20.853259 140212654778240 deprecation_wrapper.py:119] From /content/darkflow/darkflow/net/build.py:17: The name tf.train.AdagradOptimizer is deprecated. Please use tf.compat.v1.train.AdagradOptimizer instead.

W0620 09:55:20.858210 140212654778240 deprecation_wrapper.py:119] From /content/darkflow/darkflow/net/build.py:18: The name tf.train.AdagradDAOptimizer is deprecated. Please use tf.compat.v1.train.AdagradDAOptimizer instead.

W0620 09:55:20.860316 140212654778240 deprecation_wrapper.py:119] From /content/darkflow/darkflow/net/buil

In [0]:
options = {"model": "cfg/myyolov2-voc.cfg",
           "load": "backup/myyolov2-voc-20000.weights",
           "threshold": 0.23
           }
tfnet = TFNet(options)
class_names = ['0',
               '1',
               '2',
               '3',
               '4',
               '5',
               '6',
               '7',
               '8',
               '9',
               "'",
               '/',
               '.',
               'yy',
               'mm',
               'dd']
num_classes = len(class_names)
class_colors = []
for i in range(0, num_classes):
    hue = 255*i/num_classes
    col = np.zeros((1, 1, 3)).astype("uint8")
    col[0][0][0] = hue
    col[0][0][1] = 128
    col[0][0][2] = 255
    cvcol = cv2.cvtColor(col, cv2.COLOR_HSV2BGR)
    col = (int(cvcol[0][0][0]), int(cvcol[0][0][1]), int(cvcol[0][0][2]))
    class_colors.append(col)

/content/darkflow/darkflow/dark/darknet.py:54: UserWarning: ./cfg/myyolov2-voc-20000.cfg not found, use cfg/myyolov2-voc.cfg instead
  cfg_path, FLAGS.model))
W0620 09:55:20.920328 140212654778240 deprecation_wrapper.py:119] From /content/darkflow/darkflow/net/build.py:105: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0620 09:55:20.926715 140212654778240 deprecation_wrapper.py:119] From /content/darkflow/darkflow/net/ops/baseop.py:70: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

W0620 09:55:20.927877 140212654778240 deprecation_wrapper.py:119] From /content/darkflow/darkflow/net/ops/baseop.py:71: The name tf.get_variable is deprecated. Please use tf.compat.v1.get_variable instead.

W0620 09:55:20.940935 140212654778240 deprecation_wrapper.py:119] From /content/darkflow/darkflow/net/ops/baseop.py:84: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default i

Parsing cfg/myyolov2-voc.cfg
Loading backup/myyolov2-voc-20000.weights ...
Successfully identified 202622264 bytes
Finished in 0.015582799911499023s

Building net ...
Source | Train? | Layer description                | Output size
-------+--------+----------------------------------+---------------
       |        | input                            | (?, 416, 416, 3)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 416, 416, 32)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 208, 208, 32)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 208, 208, 64)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 104, 104, 64)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 104, 104, 128)
 Load  |  Yep!  | conv 1x1p0_1  +bnorm  leaky      | (?, 104, 104, 64)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 104, 104, 128)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 52, 52, 128)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 52, 

W0620 09:55:24.089865 140212654778240 deprecation.py:506] From /content/darkflow/darkflow/net/ops/convolution.py:28: calling extract_image_patches (from tensorflow.python.ops.array_ops) with ksizes is deprecated and will be removed in a future version.
Instructions for updating:
ksizes is deprecated, use sizes instead


 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 13, 13, 1024)
 Load  |  Yep!  | concat [16]                      | (?, 26, 26, 512)
 Load  |  Yep!  | conv 1x1p0_1  +bnorm  leaky      | (?, 26, 26, 64)
 Load  |  Yep!  | local flatten 2x2                | (?, 13, 13, 256)
 Load  |  Yep!  | concat [27, 24]                  | (?, 13, 13, 1280)


W0620 09:55:24.814077 140212654778240 deprecation_wrapper.py:119] From /content/darkflow/darkflow/net/build.py:145: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 13, 13, 1024)
 Load  |  Yep!  | conv 1x1p0_1    linear           | (?, 13, 13, 105)
-------+--------+----------------------------------+---------------
Running entirely on CPU
Finished in 5.8761444091796875s



In [0]:
# バウンディングボックスを描画して出力
def rectangle(imgcv, result):
    for item in result:
        tlx = item['topleft']['x']
        tly = item['topleft']['y']
        brx = item['bottomright']['x']
        bry = item['bottomright']['y']
        label = item['label']
        conf = item['confidence']
        if conf > 0.1:
            for i in class_names:
                if label == i:
                    class_num = class_names.index(i)
                    break
            # 枠の作成
            cv2.rectangle(imgcv, (tlx, tly), (brx, bry),
                          class_colors[class_num], 2)
            # ラベルの作成
#             text = label + " " + ('%.2f' % conf)
            text = label
            cv2.rectangle(imgcv, (tlx, tly - 15), (tlx + 10,
                                                   tly + 5), class_colors[class_num], -1)
            cv2.putText(imgcv, text, (tlx, tly),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)
    return imgcv


# 予測精度の計算
def calc_match(predict, answer):
    g = difflib.Differ().compare(predict.split(" "),
                                 answer.split(" "))
    M = 0
    for line in g:
        if re.match(" ", line):
            M += 1
    T = len(predict.split(" ")) + len(answer.split(" "))
    return 2*M/T


# 編集距離の計算
def diff_length(A, B):
    g = difflib.Differ().compare(A.split(" "),
                                 B.split(" "))
    M = 0
    for line in g:
        if not re.match(" ", line):
            M += 1
    return M

In [0]:
# 文字検出
def predict_date(result):
    flag = False
    flag_year = False
    flag_month = False
    flag_day = False
    s = []
    spaces = []
    x_right = 0
    y_top = 0
    diff_top = 0
    p = 1
# バウンディングボックスが重なるかどうか用
    x_left_box = 0
    x_right_box = 0
    i = 0
    for data in result:
        l = data["label"]
        #print("現在：", s, spaces)
        x_width = int(data["bottomright"]["x"]) - int(data["topleft"]["x"])
        y_width = int(data["bottomright"]["y"]) - int(data["topleft"]["y"])
        if len(s) > 1:
            if flag_year or flag_month or flag_day:
                pass
#             文字間隔が前後で違う場合，マージンを変更して区切る
            elif re.match("[0-9]", s[-2]) and re.match("[0-9]", s[-1]) and re.match("[0-9]", l):
                m = int(data["topleft"]["x"]) - result[i-1]["bottomright"]["x"]
                if c_margin > 15 and 16 > c_margin - abs(m) > 7 and result[i-1]["label"] != "'":
                    if s[-2] != "1" or s[-1] != "1":
                        #print("左に隙間があったので分割します", l)
                        s.insert(-1, " ")
                        spaces.insert(-1, margin)
                    elif c_margin - abs(m) > 11:
                        #print("11だけど左に隙間があったので分割します")
                        s.insert(-1, " ")
                        spaces.insert(-1, margin)
                elif m > 15 and 16 > abs(m)-c_margin > 7:
                    if not s[-1] == l:
                        if s[-2] != "2" and s[-1] != "0":
                            #print("右に隙間があったので分割します")
                            s.append(" ")
                            spaces.append(margin)
                    else:
                        x_right = x_right + 10

        if i == 0:
            c_margin = int(data["topleft"]["x"])
        else:
            c_margin_pre = c_margin
            c_margin = int(data["topleft"]["x"]) - \
                result[i-1]["bottomright"]["x"]
        #print(l, "横：", x_width, "縦：", y_width,"c_margin:", c_margin, "char", l)

#         急に高さの違う単語が現れたら除外
        if diff_top > 35 and abs(int(data["topleft"]["y"]) - y_top) > 35 and len(s) > 0:
            if s[-1] == result[i-1]["label"]:
                #print("途中で高さの違う単語")
                del s[-1], spaces[-1]
        if re.match("7", l) and y_width < 27:
            #print("boxが小さい")
            continue
        elif re.match("[0-9]", l) and y_width < 22:
            #print("boxが小さい")
            continue

#         ボックスが重なって検出した場合の処理
        diff_left = abs(int(data["topleft"]["x"]) - x_left_box)
        diff_right = abs(int(data["bottomright"]["x"]) - x_right_box)
        diff_top = abs(int(data["topleft"]["y"]) - y_top)
        #print("ボックス差分", diff_left, diff_right, diff_top)

#     重なったボックスは確率の高い方を選択
        if diff_left < 8 or diff_right < 8 and l != ".":
            if len(s) > 0 and data["confidence"] - p > 0:
                s[-1] = l
                #print("boxかぶりあり", l)
                x_left = int(data["topleft"]["x"])
                x_right = int(data["bottomright"]["x"])
                y_top = int(data["topleft"]["y"])
                y_bottom = int(data["bottomright"]["y"])
                continue
            elif len(s) == 0 and l != "'":
                s.append(l)
                spaces.append(0)
                x_left = int(data["topleft"]["x"])
                x_right = int(data["bottomright"]["x"])
                y_top = int(data["topleft"]["y"])
                y_bottom = int(data["bottomright"]["y"])
                continue
            else:
                #print("boxかぶりあり", l)
                continue
        elif len(s) > 0:
            if s[-1] == l:
                if diff_left < 13 or diff_right < 13:
                    s[-1] = l
                    #print("boxかぶりあり", l)
                    x_left = int(data["topleft"]["x"])
                    x_right = int(data["bottomright"]["x"])
                    y_top = int(data["topleft"]["y"])
                    y_bottom = int(data["bottomright"]["y"])
                    continue

#       'と2文字目がちょっと離れている場合
        if flag == "'" and len(s) == 0:
            margin = c_margin
            if 61 > margin > 30 and l not in ["yy", "mm", "dd"]:
                #print("9を補完しました")
                s.append("9")
                spaces.append(0)
                #print("flagがFalseになりました")
                flag = False
#           あまり離れていないとき
            elif margin <= 30:
                flag = "'9"
            else:
                #print("flagがFalseになりました")
                flag = False
        elif flag == "'" and result[-1]["label"] == l and result[-2]["label"] == "'":
            flag = "'end"
#       'X_Xとなったとき   (6 2')(1'96)(XX ')
        elif flag == "'" and len(s) == 3:
            if re.match("[0-9]", l) and re.match("[8-9]", s[-1]):
                pass
            elif s[-2] == " " and spaces[-2]+c_margin_pre < 45:
                #print('9を最初に追加しました')
                s.insert(0, "9")
                spaces.insert(0, 0)

#         20  .XX
        if flag == "." and len(s) > 1:
            if s[0] == "2" and s[1] == "0":
                if not re.search("201[0-9]", "".join(s)[:4]):
                    #print("12を追加しました")
                    s.insert(2, "2")
                    spaces.insert(2, 0)
                    s.insert(2, "1")
                    spaces.insert(2, 0)
                    flag = False
#             2__0.
            elif s[0] == "2" and s[1] == " " and s[2] == "0":
                if len(s) == 3:
                    #print("10を追加しました")
                    s = ["2", "0", "1", "0"]
                    spaces = [0, 0, 0, 0]
                    flag = False

        if flag == "/" and l == "/":
            flag == "/2"

#         次の文字と大きな高さの差があるとき
        if diff_top > 35 and len(s) > 0:
            #print("高さに差がある")
            if len(s) < 3:
                s = ["h"]
                spaces.append(0)
            else:
                continue

        x_left = int(data["topleft"]["x"])
        y_top = int(data["topleft"]["y"])
        y_bottom = int(data["bottomright"]["y"])

        if l == "'":
            flag = "'"
#             マージンを無理やり作成する用
            if len(s) == 0:
                x_right = x_right - 50
            #print("'　を検出しました", x_right)
            #print(x_left, x_right)
        elif l == ".":
            flag = "."
            #print("flagが．になりました")
            #print(x_left, x_right)
        elif l == "/":
            flag = "/"
            #print(x_left, x_right)
            if c_margin > 40 and len(s) == 2:
                if s[0] == "2" and s[1] == "0":
                    #print("10を追加しました")
                    s.extend(["1", "0"])
                    spaces.extend([0, 0])

        elif l == "yy":
            #print("flag_yearがyyになりました")
            flag_year = "yy"
#            '9X年のとき
            if flag == "'9" and s[0] == "9" and len(s) == 1:
                #print("4を加えました")
                s.append("4")
                spaces.append(0)
#             X年のとき
            elif len(s) == 1:
                #print("9を加えました")
                s.insert(0, "9")
                spaces.insert(0, 0)
            #print(x_left, x_right)
        elif l == "mm":
            flag_month = "mm"
            #print("flag_monthがmmになりました")
            #print(x_left, x_right)
        elif l == "dd":
            flag_day = "dd"
            #print("flag_dayがddになりました")
            #print(x_left, x_right)
        else:
            # 一つ前の文字との間隔で区切りを処理していく
            margin = x_left - x_right

            if x_right <= 0:
                #print("first", l)
                s.append(l)
                spaces.append(0)
            elif len(s) == 0:
                #print("firstその２", l)
                s.append(l)
                spaces.append(0)
            else:
                # 1はマージンが大きめ
                if l == "1" and s[-1] == "1":
                    #print("A")
                    if margin > 26:
                        #print("space あり")
                        s.append(" ")
                        spaces.append(margin)
                elif re.match("[0|2-9]", l) and s[-1] == "1":
                    #print("B")
                    if margin > 20:
                        #print("space あり")
                        s.append(" ")
                        spaces.append(margin)
                elif re.match("[0|2-9]", s[-1]) and l == "1":
                    #print("C")
                    if margin > 22:
                        #print("space あり")
                        s.append(" ")
                        spaces.append(margin)
                elif re.match("[0-6|8-9]", s[-1]) and l == "7":
                    #print("D")
                    if margin > 19:
                        #print("space あり")
                        s.append(" ")
                        spaces.append(margin)
                elif s[-1] == "9" and l == "0":
                    #print("E")
                    if margin > 14:
                        #print("space あり")
                        s.append(" ")
                        spaces.append(margin)
                elif s[-1] == "0" and l == "0":
                    #print("F")
                    if margin > 23:
                        #print("space あり")
                        s.append(" ")
                        spaces.append(margin)
                elif len(s) == 1 and flag == "'" and result[0]["label"] != "'":
                    #print("G")
                    if margin > 22:
                        #print("space あり")
                        s.append(" ")
                        spaces.append(margin)
                elif len(s) == 1 and flag == "'9":
                    #print("I")
                    if margin > 22:
                        #print("space あり")
                        s.append(" ")
                        spaces.append(margin)
                elif margin > 16:
                    #print("H")
                    #print("space あり")
                    s.append(" ")
                    spaces.append(margin)
                    if flag == "'end":
                        s.append("9")
                        spaces.append(0)
                s.append(l)
                spaces.append(0)
                c_margin = int(data["topleft"]["x"]) - x_right
            #print(x_left, x_right, margin)
#             ひとつ前の文字の右x軸
            x_right = int(data["bottomright"]["x"])
            x_right_box = int(data["bottomright"]["x"])
            x_left_box = int(data["topleft"]["x"])
            p = data["confidence"]
        i += 1

#     'X 1文字の場合
    if flag == "'9" and len(s) == 1:
        #print("8を末尾に加えました")
        s.append("8")
        spaces.append(0)
#     'X_XX 年が1文字の場合
    elif flag == "'9" and s[1] == " " and s[0] == "9":
        #print("8を2番目に加えました")
        s.insert(1, "8")
        spaces.insert(1, 0)
    elif flag == "'9" and s[1] == " " and re.search("[4-7]", s[0]):
        #print("9を1番目に加えました")
        s.insert(0, "9")
        spaces.insert(0, 0)
#     XX X 'X 後ろの場合の年が一文字の場合
    elif flag == "'9" and len(s) > 2 and s[-1] == "9":
        if result[-2]["label"] == "'":
            #print("7を末尾に加えました")
            s.append("7")
            spaces.append(0)
    elif len(s) > 2:
        if flag == "'" and s[-2] == "9" and s[-3] == "1":
            #print("spaceを加えました")
            s.insert(-2, " ")
            spaces.insert(-2, margin)

#     /があったら
    if flag == "/" or flag == "/2":
        if " " in s:
            tmp = "".join(s).split(" ")
            if len(tmp[0]) == 2:
                if tmp[0][0] == "2" and re.search("[0|1]", tmp[0][1]):
                    tmp[0] = "2010"
                    s = [c for c in " ".join(tmp)]
            elif len(tmp) == 4:
                if tmp[0] == "2" and tmp[1] == "0":
                    tmp[0] = "2010"
                    del tmp[1]
                    s = [c for c in " ".join(tmp)]
            elif len(tmp[0]) == 1:
                if tmp[0] in ["0", "1"]:
                    tmp[0] = "2010"
                    s = [c for c in " ".join(tmp)]
            for i, d in enumerate(tmp):
                if len(d) == 1:
                    tmp[i] = "0"+d
            s = [c for c in " ".join(tmp)]
        else:
            if len(s) == 1:
                s = ["0"+s[0]]

#     XX_XXXだったらXX_X_XXに
    if len(s) > 2:
        if re.match("[0-1]", s[0]) and re.match("[0-9]", s[1]):
            if spaces[2] > 25:
                tmp = "".join(s).split(" ")
                if len(tmp) == 2:
                    if len(tmp[1]) == 3:
                        tmp[1] = tmp[1][0]+" "+tmp[1][1:]
                        s = [c for c in " ".join(tmp)]

#     0_00
    tmp = "".join(s).split(" ")
    if len(tmp) == 2:
        if np.all([True if x == "0" else False for x in tmp[1]]):
            s = []
            spaces = []

#     0_があったら03_に
    if len(s) > 1:
        if s[0] == "0" and s[1] == " ":
            #print("3を加えました")
            s.insert(1, "3")
            spaces.insert(1, 0)

#     X_XだったらXにする
    if len(s) == 3:
        if s[0] == s[-1]:
            s = [s[0]]
            spaces = [0]

#     月や日があり，X_X_Xだったら
    if flag_month == "mm" or flag_day == "dd":
        tmp = "".join(s).split(" ")
        if len(tmp) == 3:
            #print(tmp)
            if len(tmp[0]) == 1:
                #print("9を最初に加えました")
                s.insert(0, "9")
                spaces.insert(0, 0)
#         XX_X__X
            elif len(tmp[0]) == 2:
                if 80 > spaces[4] > 50 and tmp[1] == "1" and 50 > spaces[2]:
                    #print("0を加えました")
                    s.insert(4, "0")
                    spaces.insert(4, 0)

#     XX__X月のとき
    if flag_month == "mm" and len(s) > 1:
        tmp = "".join(s).split(" ")
        if len(tmp[0]) == 2 and len(tmp[0]) == 1:
            if 80 > spaces[2] > 50:
                #print("1を加えました")
                s.insert(3, "1")
                spaces.insert(3, 0)

#     XX__X日のとき
    if flag_day == "dd" and len(s) > 1:
        tmp = "".join(s).split(" ")
        if len(tmp) > 1:
            if len(tmp[-2]) == 2 and 75 > spaces[-2] > 50 and not flag_month:
                #print("1を加えました")
                s.insert(-1, "1")
                spaces.insert(-1, 0)

#     高さに差があったら
    if len(s) > 0:
        if s[0] == "h":
            del s[0]
            if len(s) > 0:
                if s[0] == " ":
                    del s[0]
                s = "".join(s)
                s = s.split(" ")

    #print("flag: ", flag)
    #print("flag_year: ", flag_year)
    #print("flag_month: ", flag_month)
    #print("flag_day: ", flag_day)
    #print(s)
    #print(spaces)
    predict = "".join(s)

#     1文字の場合，日付無しを誤認する場合が多い
    if len(predict) == 1:
        predict = ""

    predict = reshape_predict(predict)

    return predict, spaces

In [0]:
# 予測結果の整形
def reshape_predict(predict):
#     なぜか記号が混ざっているので除外
    predict = re.sub("['|/|.|]","",predict)
#     6文字以上連続
    m = re.search("^20[0-8]{4,}", predict)
    if m:
        predict = re.sub(m.group(), m.group()[
                         :2]+"1"+m.group()[2]+" "+m.group()[3:], predict)
        #print(m.group())
#     4文字連続
    m = re.search("^[2|0|7]{4}", predict)
    if m:
        predict = re.sub(m.group(), m.group()+" ", predict)
        #print(m.group())
#     8文字連続
    m = re.search("^[0-9]{8}$", predict)
    if m:
        predict = re.sub(m.group(), m.group()[
                         :4]+" " + m.group()[4:6]+" " + m.group()[6:], predict)
        #print(m.group())
#     7文字連続
    m = re.search("^[0-9]{7}$", predict)
    if m:
        predict = re.sub(m.group(), m.group()[
                         :2]+" " + m.group()[2]+" " + m.group()[3:5]+" " + m.group()[5:], predict)
        #print(m.group())
#    6文字連続
    m = re.search("^[0-9]{6}$", predict)
    if m:
        predict = re.sub(m.group(), m.group()[
                         :2]+" " + m.group()[2:4]+" " + m.group()[4:6], predict)
        #print(m.group())
#    5文字連続
    m = re.search("^[0-9]{5}$", predict)
    if m:
        predict = re.sub(m.group(), m.group()[
                         0]+" " + m.group()[1:3]+" " + m.group()[3:5], predict)
        #print(m.group())
#    4文字連続　年
    m = re.search("[0|9][0-9][0-9][0-9]", predict)
    if m:
        predict = re.sub(m.group(), m.group()[
                         :2]+" " + m.group()[2:], predict)
        #print(m.group())
#    4文字連続　年
    m = re.search("^2[1-9][0-9][0-9] ", predict)
    if m:
        predict = re.sub(m.group(), "20"+m.group()[
                         2:], predict)
        #print(m.group())
#    年確定の場合
    m = re.search("^9[0-9] [1-9][0-3][0-9]$", predict)
    if m:
        predict = re.sub(m.group(), m.group()[
                         :4]+" " + m.group()[4:], predict)
        #print(m.group())
#    ありえない文字列
    m = re.search("[4|5|6|7][0-9]+", predict)
    if m:
        predict = re.sub(m.group(), m.group()[
                         0]+" " + m.group()[1:], predict)
        #print(m.group())
#    4文字連続　後半
    m = re.search(" [1-9][0-9][0-9][0-9]$", predict)
    if m :
        predict = re.sub(m.group(), m.group()[:3]+" "+m.group()[3:], predict)
        #print(m.group())
#    3文字以上連続　11は1の可能性が高い
    m = re.search(" 11[0-9]+", predict)
    if m:
        predict = re.sub(m.group(), " "+m.group()[2:], predict)
        #print(m.group())
#    3文字以上連続
    m = re.search(" +[8|9][0-9][0-9]+", predict)
    if m:
        predict = re.sub(m.group()[1:], m.group()[-2:], predict)
        #print(m.group())
#    分割点が3つ
    m = re.search("[0-9] [0-9] [0-9] ", predict)
    if m:
        predict = re.sub(m.group(), m.group()[:3]+m.group()[4:], predict)
        #print(m.group())
#    分割点が4つ
    m = re.search("^[0-9] [0-9] [0-9] [0-9] [0-9]$", predict)
    if m:
        if m.group()[2] == m.group()[4] and m.group()[4] == m.group()[6]:
            predict = re.sub(m.group(), m.group()[:2]+m.group()[6:], predict)
        elif m.group()[2] == m.group()[4]:
            predict = re.sub(m.group(), m.group()[:2]+m.group()[4:], predict)
        elif m.group()[4] == m.group()[6]:
            predict = re.sub(m.group(), m.group()[:4]+m.group()[6:], predict)
        #print(m.group())
#    あり得ない文字列
    m = re.search(" [4-9][0-9] ", predict)
    if m:
        predict = re.sub(m.group(), m.group()[
                         :2]+" " + m.group()[2:], predict)
        #print(m.group())
#    3文字連続　年
    m = re.search("^[8-9][0-9][0-9] ", predict)
    if m:
        predict = re.sub(m.group(), m.group()[
                         0]+" " + m.group()[1:], predict)
        #print(m.group())
#    分割点3つ
    m = re.search("^[0|8|9] [0-9] [0-9]+ [0-9]+", predict)
    if m:
        predict = re.sub(m.group(), m.group()[0]+m.group()[2:], predict)
        #print(m.group())
#    4文字連続
    m = re.search("^1[0-2][1-3][0-9] ", predict)
    if m:
        predict = re.sub(m.group(), m.group()[:2]+" "+m.group()[2:], predict)
        #print(m.group())
#    2の誤認
    m = re.search(" 122 ", predict)
    if m:
        predict = re.sub(m.group(), " 12 ", predict)
        #print(m.group())
#    3文字連続
    m = re.search(" [0|2|9][0-9][0-9]$", predict)
    if m:
        predict = re.sub(m.group(), " "+m.group()[2:], predict)
        #print(m.group())
#    3文字連続
    m = re.search(" 1[0-9][0-9]$", predict)
    if m:
        predict = re.sub(m.group(), " 1 "+m.group()[2:], predict)
        #print(m.group())
#    3文字連続
    m = re.search(" 3[0-9][0-9]$", predict)
    if m:
        predict = re.sub(m.group(), " 3 "+m.group()[2:], predict)
        #print(m.group())
#    3文字連続　20XX年
    m = re.search("^20[0-9] ", predict)
    if m:
        predict = re.sub(m.group(), m.group()[:2]+"1"+m.group()[2:], predict)
        #print(m.group())
#    3文字連続　20XX年
    m = re.search("^21[0-9] ", predict)
    if m:
        predict = re.sub(m.group(), m.group()[0]+"0"+m.group()[1:], predict)
        #print(m.group())
#    4文字連続　20XX年
    m = re.search("^211[0-9]+ ", predict)
    if m:
        predict = re.sub(m.group(), m.group()[
                         0]+"0"+m.group()[1:3]+" "+m.group()[3:], predict)
        #print(m.group())
#    4文字連続　20XX年
    m = re.search("^20[2-9][0-9] ", predict)
    if m:
        predict = re.sub(m.group(), "2010 ", predict)
        #print(m.group())
#    3文字連続　年
    m = re.search("^004 ", predict)
    if m:
        predict = re.sub(m.group(), "2004 ", predict)
        #print(m.group())
#    3文字連続　年
    m = re.search("^0[0|2][0-9] ", predict)
    if m:
        predict = re.sub(m.group(), m.group()[:2]+" "+m.group()[2:], predict)
        #print(m.group())
#    ありえない文字列
    m = re.search(" 3[2-9] ", predict)
    if m:
        predict = re.sub(m.group(), " 3 ", predict)
        #print(m.group())
#    ありえない文字列
    m = re.search("^[1|3][0-9]{2} ", predict)
    if m:
        predict = re.sub(m.group(), m.group()[0]+" "+m.group()[1:], predict)
        #print(m.group())
#    ありえない文字列
    m = re.search(" [4-9][0-9] ", predict)
    if m:
        predict = re.sub(m.group(), " "+m.group()[1]+" ", predict)
        #print(m.group())
#    ありえない文字列
    m = re.search(" [4-7][0-9]$", predict)
    if m:
        predict = re.sub(m.group(), " 2"+m.group()[2], predict)
        #print(m.group())
#    ありえない文字列
    m = re.search(" 3[2-9]$", predict)
    if m:
        predict = re.sub(m.group(), "2"+m.group()[2], predict)
        #print(m.group())
#    ありえない文字列
    m = re.search("^3[2-9] ", predict)
    if m:
        predict = re.sub(m.group(), "3 ", predict)
        #print(m.group())
#    ありえない文字列
    m = re.search("^[3-7]6 ", predict)
    if m:
        predict = re.sub(m.group(), "86 ", predict)
        #print(m.group())
#    ありえない文字列
    m = re.search("^[4-7][0-9] ", predict)
    if m:
        predict = re.sub(m.group(), m.group()[0]+" ", predict)
        #print(m.group())
#    ありえない文字列
    m = re.search("^[4-7][0-9]$", predict)
    if m:
        predict = ""
        #print(m.group())
#    ありえない文字列
    m = re.search("^3[2-9]$", predict)
    if m:
        predict = ""
        #print(m.group())
#    3文字連続のみ
    m = re.search("^[0-9]{3}$", predict)
    if m:
        predict = re.sub(m.group(), m.group()[0]+" "+m.group()[1:], predict)
        #print(m.group())
#    4文字連続のみ
    m = re.search("^1[0-9]{3}$", predict)
    if m:
        predict = re.sub(m.group(), m.group()[:2]+" "+m.group()[2:], predict)
        #print(m.group())
#    年
    m = re.search("^0[0-9] 04 ", predict)
    if m:
        predict = re.sub(m.group(), "2004 ", predict)
        #print(m.group())
#    分割点が3つ
    m = re.search("^[0-9]+ [0-9]+ [0-9]+ [0-9]+$", predict)
    if m:
        tmp = predict.split(" ")
        if len(set(tmp)) == 3:
            l_unique_order = list(dict.fromkeys(tmp))
            predict = " ".join(l_unique_order)
        elif len(set(tmp)) == 2:
            del tmp[0]
            predict = " ".join(tmp)
        #print(m.group())
#    分割点が3つ
    m = re.search("^[0-9]+ [0-9]+ [0]+ [0-9]+$", predict)
    if m:
        tmp = predict.split(" ")
        del tmp[2]
        predict = " ".join(tmp)
        #print(m.group())
#    分割点が3つ
    m = re.search("^[0-9]+ [0]+ [0-9]+ [0-9]+$", predict)
    if m:
        tmp = predict.split(" ")
        del tmp[1]
        predict = " ".join(tmp)
        #print(m.group())
#    ありえない文字列
    m = re.search("^0 ", predict)
    if m:
        predict = re.sub(m.group(), "00 ", predict)
        #print(m.group())
#    ありえない文字列
    m = re.search(" 0 ", predict)
    if m:
        predict = re.sub(m.group(), " 10 ", predict)
        #print(m.group())
#    ありえない文字列
    m = re.search(" 0$", predict)
    if m:
        predict = re.sub(m.group(), " 10", predict)
        #print(m.group())
#    特殊
    m = re.search("^00$", predict)
    if m:
        predict = ""
        #print(m.group())
#    年
    m = re.search("^201[6-8] ", predict)
    if m:
        predict = re.sub(m.group(), "200"+m.group()[3]+" ", predict)
        #print(m.group())
#    分割できそう
    m = re.search("^[8-9][0-9]1[0-2] ", predict)
    if m:
        predict = re.sub(m.group(), m.group()[:2]+" "+m.group()[2:], predict)
        #print(m.group())
#    年
    m = re.search("^[0|2][0|1|8]{2} ", predict)
    if m:
        predict = re.sub(m.group(), "200"+m.group()[2]+" ", predict)
        #print(m.group())
#    3文字連続　
    m = re.search("^[3-9][0-9]{3} ", predict)
    if m:
        predict = re.sub(m.group(), "00 ", predict)
        #print(m.group())
#    年
    m = re.search("^2 0[0-9] ", predict)
    if m:
        predict = re.sub(m.group(), "200"+m.group()[3]+" ", predict)
        #print(m.group())
#    3文字連続
    m = re.search(" 1[3-9][8|9] [0-9]$", predict)
    if m:
        predict = re.sub(m.group(), " 12 "+m.group()[3]+m.group()[5], predict)
        #print(m.group())
#    3文字連続
    m = re.search(" 1[0-2][8|9] [0-9]$", predict)
    if m:
        predict = re.sub(m.group(), " 1"+m.group()
                         [2]+" "+m.group()[3]+m.group()[5], predict)
        #print(m.group())
#    3文字連続
    m = re.search(" 1[0-2]{2} [0-9]$", predict)
    if m:
        predict = re.sub(m.group(), m.group()
                         [:3]+" "+m.group()[3]+m.group()[5], predict)
        #print(m.group())
#    3文字連続
    m = re.search(" [0-9]{3}", predict)
    if m:
        predict = re.sub(m.group(), m.group()
                         [:2]+" "+m.group()[2:], predict)
        #print(m.group())
#    0だらけ
    if len(set(predict.split(" "))) == 1:
        tmp = predict.split(" ")
        if re.search("^0+$", tmp[0]):
            predict = tmp[0]
#    ありえない文字列
    if len(predict.split(" ")) > 3:
        tmp = predict.split(" ")
        for d in tmp:
            if len(d) == 3:
                tmp.remove(d)
        predict = " ".join(tmp)
#    空白2連続
    m = re.search("  ", predict)
    if m:
        predict = re.sub(m.group(), " ", predict)
        #print(m.group())
#    先頭空文字
    m = re.match(" ", predict)
    if m:
        predict = predict[1:]
        #print(m.group())
    return predict

In [0]:
reshape_predict("9 3 10 19")

'93 10 19'

In [0]:
# 予測結果と前後のデータの予測結果を比べて整形
# 同じユーザのデータはだいたいおんなじ日付？
def predicts_reshaper(predicts, spaces, data_info):
#     同一ユーザのデータを抽出
    file_ids = list(set(re.findall("id[0-9]+", " ".join(predicts.keys()))))
    file_ids.sort()
    window = 1  # 1以外指定禁止
    new_predicts = {}
    for file_id in file_ids:
        f = re.findall(data_info+file_id +
                       "_no[0-9]+\.jpg", " ".join(predicts.keys()))
        predict_only_one_person = [predicts[n] for n in f]
        new_data = []
#         文字列パターンをカウント
        number_count_dic = {}
        for i, data in enumerate(predict_only_one_person):
            d = data.split(" ")
            if data == "":
                try:
                    number_count_dic["NULL"] += 1
                except KeyError:
                    number_count_dic["NULL"] = 1
            else:
                for x in d:
                    try:
                        number_count_dic[x] += 1
                    except KeyError:
                        number_count_dic[x] = 1
#         日付パターンをカウント
        number_date_dic = {}
        for i, data in enumerate(predict_only_one_person):
            if data == "":
                try:
                    number_date_dic["NULL"] += 1
                except KeyError:
                    number_date_dic["NULL"] = 1
            else:
                try:
                    number_date_dic[data] += 1
                except KeyError:
                    number_date_dic[data] = 1

        for i, data in enumerate(predict_only_one_person):
            new_data.append(data)
            d = data.split(" ")
            for j in range(window):
#                 前後の予測結果とのdiff距離を保存
                d_plus, d_minus = 100, 100
                if i+j+1 < len(predict_only_one_person):
                    d_plus = diff_length(data, predict_only_one_person[i+j+1])
                    tmp_plus = predict_only_one_person[i+j+1].split(" ")
                if i-j-1 >= 0:
                    d_minus = diff_length(data, predict_only_one_person[i-j-1])
                    tmp_minus = predict_only_one_person[i-j-1].split(" ")
                if d_plus == 1 and d_minus == 1:
#                     diffが1でnextが3分割できるなら，それにあわせる 05_14 and 05_14_16
                    if len(d) == 2 and len(tmp_plus) == 3:
                        if predict_only_one_person[i+j+1] == predict_only_one_person[i-j-1]:
                            new_data[-1] = predict_only_one_person[i+j+1]
#                     diffが1でnextが2分割できるなら，それにあわせる 05 and 05_14
                    elif len(d) == 1 and len(tmp_plus) == 2:
                        if predict_only_one_person[i+j+1] == predict_only_one_person[i-j-1]:
                            new_data[-1] = predict_only_one_person[i+j+1]
                elif d_plus == 1:
#                     diffが1でnextが3分割できるなら，それにあわせる 05_14 and 05_14_16
                    if len(d) == 2 and len(tmp_plus) == 3:
                        new_data[-1] = predict_only_one_person[i+j+1]
                elif d_minus == 1:
#                     diffが1でpreviousが3分割できるなら，それにあわせる 05_14 and 05_14_16
                    if len(d) == 2 and len(tmp_minus) == 3:
                        new_data[-1] = predict_only_one_person[i-j-1]
                elif d_plus == 2:
#                     2分割できて，先頭文字列が一致　05_14 and 05___16
                    if len(d) == 2 and len(tmp_plus) == 2 and d[0] == tmp_plus[0]:
                        space = [x for x in spaces[f[i]] if x != 0]
                        space_tmp = [x for x in spaces[f[i+1]] if x != 0]
                        if len(space) > 0 and len(space_tmp) > 0:
                            if space[0] - space_tmp[0] > 50:
                                d.insert(1, tmp_plus[-1])
                                new_data[-1] = " ".join(d)
                            elif space[0] - space_tmp[0] < -50:
                                d.append(tmp_plus[-1])
                                new_data[-1] = " ".join(d)
                elif d_minus == 2:
#                     2分割できて，先頭文字列が一致　05_14 and 05___16
                    if len(d) == 2 and len(tmp_minus) == 2 and d[0] == tmp_minus[0]:
                        space = [x for x in spaces[f[i]] if x != 0]
                        space_tmp = [x for x in spaces[f[i-1]] if x != 0]
                        if len(space) > 0 and len(space_tmp) > 0:
                            if space[0] - space_tmp[0] > 50:
                                d.insert(1, tmp_minus[-1])
                                new_data[-1] = " ".join(d)
                            elif space[0] - space_tmp[0] < -50:
                                d.append(tmp_minus[-1])
                                new_data[-1] = " ".join(d)
#               予測パターンが多いものに似ていたら（diff=2）
                if d_plus == 2 and len(d) == 3 and len(tmp_plus) == 3:
                    if number_date_dic[predict_only_one_person[i+j+1]] - number_date_dic[data] > 4:
                        new_data[-1] = predict_only_one_person[i+j+1]
                elif d_minus == 2 and len(d) == 3 and len(tmp_minus) == 3:
                    if number_date_dic[predict_only_one_person[i-j-1]] - number_date_dic[data] > 4:
                        new_data[-1] = predict_only_one_person[i-j-1]

#                 前後ともNULL，かつ，全体でその割合が高いとき，はさんだ予測値もNULLに（2分割まで）
                if 0 < d_plus < 4 and 0 < d_minus < 4:
                    try:
                        if predict_only_one_person[i+j+1] == predict_only_one_person[i-j-1] and predict_only_one_person[i+j+1] == "":
                            if number_count_dic["NULL"] / len(predict_only_one_person) > 0.1 and len(data) < 4:
                                new_data[-1] = predict_only_one_person[i+j+1]
                        elif number_count_dic["NULL"] / len(predict_only_one_person) > 0.4 and len(data) < 4:
                            if len(data) == 1:
                                new_data[-1] = ""
                    except KeyError:
                        pass

        for i, filename in enumerate(f):
            new_predicts[filename] = new_data[i]
    return new_predicts

## 学習データで検証

In [0]:
path = "../input/train_images/"
test_txt = "../test0.2.txt"
test_data = open(test_txt, "r")
test_filenames = [re.sub("data/images/", "", line) for line in test_data]
test_filenames = [re.sub("\n", "", line) for line in test_filenames]
test_data.close()
test_filenames.sort()

In [0]:
# 学習データすべて
# path = "../input/train_anns/"
# test_filenames = os.listdir(path)
# test_filenames.sort()
# test_filenames = [re.sub("json", "jpg", filename) for filename in test_filenames]
# path = "../input/train_images/"

In [0]:
Accracy = []
predicts = {}
spaces = {}
n = 1
for filename in test_filenames[0:]:
    label_path = "../input/train_anns/"+re.sub("jpg", "json", filename)
    df_label = pd.read_json(label_path)
    imgcv = cv2.imread(path+filename)
    result = tfnet.return_predict(imgcv)
    result = sorted(result, key=lambda x:x['topleft']["x"])
    answer = df_label.at[0, "label"]
    predict, space = predict_date(result)
    acc = calc_match(predict, answer)
    Accracy.append(acc)
    predicts[filename] = predict
    spaces[filename] = space
    print(n,"trial", answer,"......", predict,"......", acc, filename)
#     if n % 30 == 0:
#         break
    n += 1
#     if acc < 1:
#         imgcv = rectangle(imgcv, result)
#         plt.imshow(cv2.cvtColor(imgcv, cv2.COLOR_BGR2RGB))
#         plt.show()

1 trial 93 7 30 ...... 93 7 30 ...... 1.0 train_id0003_no0033.jpg
2 trial 93 7 30 ...... 93 7 30 ...... 1.0 train_id0003_no0035.jpg
3 trial 12 8 98 ...... 12 8 98 ...... 1.0 train_id0004_no0009.jpg
4 trial 12 8 98 ...... 12 8 98 ...... 1.0 train_id0004_no0010.jpg
5 trial 98 9 22 ...... 98 9 22 ...... 1.0 train_id0004_no0018.jpg
6 trial 98 9 23 ...... 9 23 ...... 0.8 train_id0004_no0019.jpg
7 trial 98 9 23 ...... 98 9 23 ...... 1.0 train_id0004_no0021.jpg
8 trial 98 3 1 ...... 98 3 1 ...... 1.0 train_id0004_no0063.jpg
9 trial 98 3 28 ...... 98 3 28 ...... 1.0 train_id0004_no0066.jpg
10 trial 97 4 19 ...... 97 4 ...... 0.8 train_id0004_no0068.jpg
11 trial 89 11 3 ......  ...... 0.0 train_id0005_no0001.jpg
12 trial 89 10 16 ...... 89 16 ...... 0.8 train_id0005_no0075.jpg
13 trial 6 12 95 ...... 6 95 ...... 0.8 train_id0006_no0065.jpg
14 trial 6 12 95 ...... 6  ...... 0.4 train_id0006_no0067.jpg
15 trial 90 5 28 ...... 90 5 28 ...... 1.0 train_id0007_no0069.jpg
16 trial 11 24 98 ...... 1 1

In [0]:
# thresh0.23 weights20000　0.6984
print("平均一致率：", np.average(Accracy))

平均一致率： 0.6984152752009894


In [0]:
# 予測結果の再整形
data_info = "train_"
new_predicts = predicts_reshaper(predicts, spaces, data_info)
new_predicts = predicts_reshaper(new_predicts, spaces, data_info)

In [0]:
# # 比較
# for filename in predicts.keys():
#     label_path = "../input/train_anns/"+re.sub("jpg", "json", filename)
#     df_label = pd.read_json(label_path)
#     answer = df_label.at[0, "label"]
#     if predicts[filename] != new_predicts[filename]:
#         print(filename,answer,"..." ,predicts[filename],"...", new_predicts[filename])

In [0]:
# 精度の再算出
Accracy = []
n = 1
for filename in predicts.keys():
    label_path = "../input/train_anns/"+re.sub("jpg", "json", filename)
    df_label = pd.read_json(label_path)
    result = sorted(result, key=lambda x:x['topleft']["x"])
    answer = df_label.at[0, "label"]
    predict = new_predicts[filename]
    acc = calc_match(predict, answer)
    Accracy.append(acc)
    print(n,"trial", answer,"......", predict,"......", acc, filename)
    n += 1

1 trial 93 7 30 ...... 93 7 30 ...... 1.0 train_id0003_no0033.jpg
2 trial 93 7 30 ...... 93 7 30 ...... 1.0 train_id0003_no0035.jpg
3 trial 12 8 98 ...... 12 8 98 ...... 1.0 train_id0004_no0009.jpg
4 trial 12 8 98 ...... 12 8 98 ...... 1.0 train_id0004_no0010.jpg
5 trial 98 9 22 ...... 98 9 22 ...... 1.0 train_id0004_no0018.jpg
6 trial 98 9 23 ...... 98 9 23 ...... 1.0 train_id0004_no0019.jpg
7 trial 98 9 23 ...... 98 9 23 ...... 1.0 train_id0004_no0021.jpg
8 trial 98 3 1 ...... 98 3 1 ...... 1.0 train_id0004_no0063.jpg
9 trial 98 3 28 ...... 98 3 28 ...... 1.0 train_id0004_no0066.jpg
10 trial 97 4 19 ...... 97 4 ...... 0.8 train_id0004_no0068.jpg
11 trial 89 11 3 ......  ...... 0.0 train_id0005_no0001.jpg
12 trial 89 10 16 ...... 89 16 ...... 0.8 train_id0005_no0075.jpg
13 trial 6 12 95 ...... 6 95 ...... 0.8 train_id0006_no0065.jpg
14 trial 6 12 95 ...... 6  ...... 0.4 train_id0006_no0067.jpg
15 trial 90 5 28 ...... 90 5 28 ...... 1.0 train_id0007_no0069.jpg
16 trial 11 24 98 ...... 

In [0]:
# thresh0.23 weights20000 new　 0.700
print("平均一致率：", np.average(Accracy))

平均一致率： 0.7001468769325911


## テストデータの予測

In [0]:
path = "../input/test_images/"
test_filenames = os.listdir(path)
test_filenames.sort()

In [26]:
Accracy = []
predicts = {}
spaces = {}
n = 1
for filename in test_filenames[0:]:
    label_path = "../input/train_anns/"+re.sub("jpg", "json", filename)
    imgcv = cv2.imread(path+filename)
    result = tfnet.return_predict(imgcv)
    result = sorted(result, key=lambda x:x['topleft']["x"])
    predict, space = predict_date(result)
    predicts[filename] = predict
    spaces[filename] = space
    print(n,"回目",filename, predict)
#     if predict != "":
#         imgcv = rectangle(imgcv, result)
#         plt.imshow(cv2.cvtColor(imgcv, cv2.COLOR_BGR2RGB))
#         plt.show()
#     if n % 100 == 0:
#         break
    n += 1

1 回目 test_id0001_no0001.jpg 94 9
2 回目 test_id0001_no0002.jpg 
3 回目 test_id0001_no0003.jpg 94 8 9
4 回目 test_id0001_no0004.jpg 94 8 9
5 回目 test_id0001_no0005.jpg 
6 回目 test_id0001_no0006.jpg 
7 回目 test_id0001_no0007.jpg 
8 回目 test_id0001_no0008.jpg 94 8
9 回目 test_id0001_no0009.jpg 94 8 9
10 回目 test_id0001_no0010.jpg 98 8 9
11 回目 test_id0001_no0011.jpg 
12 回目 test_id0001_no0012.jpg 
13 回目 test_id0001_no0013.jpg 94 8
14 回目 test_id0001_no0014.jpg 
15 回目 test_id0001_no0015.jpg 
16 回目 test_id0001_no0016.jpg 
17 回目 test_id0001_no0017.jpg 
18 回目 test_id0001_no0018.jpg 
19 回目 test_id0001_no0019.jpg 
20 回目 test_id0001_no0020.jpg 
21 回目 test_id0001_no0021.jpg 
22 回目 test_id0001_no0022.jpg 
23 回目 test_id0001_no0023.jpg 
24 回目 test_id0001_no0024.jpg 
25 回目 test_id0001_no0025.jpg 
26 回目 test_id0001_no0026.jpg 
27 回目 test_id0001_no0027.jpg 
28 回目 test_id0001_no0028.jpg 
29 回目 test_id0001_no0029.jpg 
30 回目 test_id0001_no0030.jpg 
31 回目 test_id0001_no0031.jpg 
32 回目 test_id0001_no0032.jpg 
33 回目 test_id

In [0]:
# df = pd.DataFrame(list(predicts.items()), columns=['key', 'value'])

In [0]:
# df.to_csv("/content/gdrive/My Drive/Python3/fujifilm/data/result_yolo_20000_0.23.csv", header=False, index=False)

In [0]:
# 予測結果の再整形
data_info = "test_"
new_predicts = predicts_reshaper(predicts, spaces, data_info)
new_predicts = predicts_reshaper(new_predicts, spaces, data_info)

In [0]:
# 整形前後の比較
# for filename in predicts.keys():
#     if predicts[filename] != new_predicts[filename]:
#         print(filename,predicts[filename],"...", new_predicts[filename])

In [0]:
df = pd.DataFrame(list(new_predicts.items()), columns=['key', 'value'])

In [0]:
from google.colab import files
df.to_csv('result_yolo_20000_0.23_new.csv', index=False, header=None)
files.download('result_yolo_20000_0.23_new.csv')

In [0]:
df.to_csv("/content/gdrive/My Drive/Python3/fujifilm/data/result_yolo_20000_0.23_new.csv", header=False, index=False)